In [ ]:
import numpy as np
import os
import pandas as pd
import pickle as pkl
from scipy.stats.stats import pearsonr

In [ ]:
# combine results into a single dataframe
res = {"embedding": [], "model": [], "k": [], "trn_mae": [], "trn_r2": [], "val_mae": [], "val_r2": [], "tst_mae": [], "tst_r2": []}

name = "../results/ncv"

for emb in os.listdir(f"../{name}/"):
    for model in os.listdir(f"../{name}/{emb}"):
        df = pd.read_csv(f"../{name}/{emb}/{model}/nCV_results.csv")
        res["embedding"] += [emb] * len(df)
        res["model"] += [model] * len(df)
        res["k"] += df.k.tolist()
        res["trn_mae"] += df.trn_mae.tolist()
        res["trn_r2"] += df.trn_r2.tolist()
        res["val_mae"] += df.val_mae.tolist()
        res["val_r2"] += df.val_r2.tolist()
        res["tst_mae"] += df.tst_mae.tolist()
        res["tst_r2"] += df.tst_r2.tolist()

res_df = pd.DataFrame(res)
heatmap_res = res_df.groupby(["model", "embedding"]).mean()[["trn_r2", "val_r2", "tst_r2"]]

display(heatmap_res.sort_values("tst_r2", ascending=False))

In [ ]:
# scaler class to scale predictions
class MinMaxScaler:

    def __init__(self, y):
        self.min_y = np.min(y)
        self.max_y = np.max(y)
    
    def transform(self, y):
        y_scaled = (y - self.min_y)/(self.max_y - self.min_y)
        return y_scaled

    def invert(self, y):
        y_orig = y*(self.max_y - self.min_y) + self.min_y
        return y_orig

In [ ]:
# dictionary containing errors for each model with appropriate R2
error_results = {
    "Model": [],
    "Encoding": [],
    "Predictions": [],
    "Actual": [],
    "Error": [],
    "Mean-error": []
}

for emb in os.listdir(f"../{name}/"):
    for model in os.listdir(f"../{name}/{emb}"):
        df = pd.read_csv(f"../{name}/{emb}/{model}/nCV_results.csv")
        if np.mean(df["tst_r2"].to_numpy()) > 0.17: 
            skl_model = pkl.load(open(f"../{name}/{emb}/{model}/final_model.sav", "rb"))

            raw_ic50 = np.load("../data/data/joint_peptide_channel_encodings/IC50_raw.npy")   
            scaler = MinMaxScaler(raw_ic50)
            y_scaled = scaler.transform(raw_ic50)

            seqs = np.load(f"../data/data/joint_peptide_channel_encodings/{emb}.npy")

            preds = skl_model.predict(seqs)

            error_results["Model"].append(model)
            error_results["Encoding"].append(emb)
            error_results["Predictions"].append(preds)
            error_results["Actual"].append(y_scaled)
            error_results["Error"].append(y_scaled - preds)
            error_results["Mean-error"].append(np.mean(np.abs(y_scaled - preds)))

In [6]:
def start_ensemble(exclude_idx, actual_ls, pred_ls, error_ls):
    best_corr_pair = (None, None)
    best_corr = 1
    for i in range(len(error_ls)):
        for j in range(i):
            if i in exclude_idx or j in exclude_idx:
                continue
            corr = pearsonr(error_ls[i], error_ls[j])
            if corr < best_corr:
                best_corr = corr
                best_corr_pair = (i, j)
    ens_pred = pred_ls[best_corr_pair[0]] + pred_ls[best_corr_pair[1]]/2
    ens_error = actual_ls - ens_pred
    mean_error = np.mean(np.abs(ens_error))
    return best_corr_pair, best_corr, mean_error
        
def add_model(exclude_idx, actual_ls, pred_ls, ens_idx, error_ls):
    best_corr_add = None
    best_corr = 1
    for i in range(len(error_ls)):
        if i in exclude_idx:
            continue
        corr = pearsonr(error_ls[i], error_ls[j])
        if corr < best_corr:
            best_corr = corr
            best_corr_add = i
    ens_preds = pred_ls[best_corr_add]
    ens_idx += [best_corr_add]
    for idx in ens_idx:
        ens_preds = ens_preds + pred_ls[idx]
    ens_preds /= len(ens_preds) + 1
    ens_error = actual_ls - ens_preds
    mean_error = np.mean(np.abs(ens_error))
    return ens_idx, best_corr, mean_error, best_corr_add

In [ ]:
res = {
    "ens_idx": [],
    "ens_error": [],
    "corr_of_added": []
}
# excluding GBT with ST-score due to outlying train score
excl_ls = [4]
ens_idx, best_corr, ens_err = start_ensemble(
    excl_ls, 
    error_results["Actual"][0],
    error_results["Predictions"],
    error_results["Error"],
)

res["ens_error"].append(ens_idx)
res["ens_error"].append(ens_err)
res["corr_of_added"].append(best_corr)

# while there are models remaining
while len(excl_ls) < len(error_results["Actual"]):
    ens_idx, best_corr, ens_err, added_idx = add_model(
        excl_ls, 
        error_results["Actual"][0], 
        error_results["Predictions"], 
        ens_idx, 
        error_results["Error"]
    )
    excl_ls += [added_idx]
    
    res["ens_error"].append(ens_idx)
    res["ens_error"].append(ens_err)
    res["corr_of_added"].append(best_corr)